# Preprocess

In [ ]:
!pip install niapy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 176 kB 7.4 MB/s 


In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import gdown

In [ ]:
!gdown --id 1OLnlno2zp6HXmIxNdNcK6l3hidH4ttVr
!gdown --id 1cqgF0TDS6-wZql4NayQvPNvJx9SBf_CD

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1OLnlno2zp6HXmIxNdNcK6l3hidH4ttVr
To: /content/boston_train.xlsx
100% 34.4k/34.4k [00:00<00:00, 42.2MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1cqgF0TDS6-wZql4NayQvPNvJx9SBf_CD
To: /content/boston_test.xlsx
100% 13.1k/13.1k [00:00<00:00, 17.9MB/s]


In [ ]:
train = pd.read_excel('boston_train.xlsx')
test = pd.read_excel('boston_test.xlsx')

In [ ]:
train.isnull().sum()

CRIM       0
ZN         0
INDUS      0
CHAS       0
NOX        0
RM         0
AGE        0
DIS        0
RAD        0
TAX        0
PTRATIO    0
B          0
LSTAT      0
PRICE      0
dtype: int64

In [ ]:
scaler = StandardScaler()

In [ ]:
X_train = train[['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']]
y_train = train[['PRICE']]
X_test = test[['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']]
y_test = test[['PRICE']]

In [ ]:
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [ ]:
print(len(X_train))
print(len(X_test))

404
102


# HybridBat

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVR

from niapy.problems import Problem
from niapy.task import OptimizationType, Task
from niapy.algorithms.modified import HybridBatAlgorithm


def get_hyperparameters(x):
    """Get hyperparameters for solution `x`."""
    print(x)
    C=int(5 + x[0] * 10)
    degree=int(5 + x[1] * 10)
    kernels=('poly', 'rbf', 'sigmoid')
    kernel=kernels[int(x[2] * 2)]
    gammas=('scale', 'auto')
    gamma=gammas[int(x[3])]

    params = {
        'C': C,
        'degree': degree,
        'kernel' : kernel,
        'gamma': gamma,
    }

    return params


def get_regressor(x):
    """Get classifier from solution `x`."""
    params = get_hyperparameters(x)
    return SVR(**params)

In [ ]:
class SVRHyperparameterOptimization(Problem):
    def __init__(self, X_train, y_train):
        super().__init__(dimension=4, lower=0, upper=1)
        self.X_train = X_train
        self.y_train = y_train

    def _evaluate(self, x):
        model = get_regressor(x)
        scores = cross_val_score(model, self.X_train, self.y_train, cv=2, n_jobs=-1)
        return scores.mean()

In [ ]:
problem = SVRHyperparameterOptimization(X_train, y_train)

# We will be running maximization for 200 iters on `problem`
task = Task(problem, max_iters=200, optimization_type=OptimizationType.MAXIMIZATION)

algorithm = HybridBatAlgorithm(population_size=12, seed=1234)
best_params_HB, best_accuracy = algorithm.run(task)

print('Best parameters:', get_hyperparameters(best_params_HB))

[0.97669977 0.38019574 0.92324623 0.26169242]
[0.31909706 0.11809123 0.24176629 0.31853393]
[0.96407925 0.2636498  0.44100612 0.60987081]
[0.8636213  0.86375767 0.67488131 0.65987435]
[0.7357577  0.22275366 0.17206618 0.87041497]
[0.06013866 0.68368891 0.67123802 0.61101798]
[0.06013731 0.97776927 0.43895163 0.53259502]
[0.00313229 0.25126711 0.85849044 0.42529835]
[0.73581899 0.92204322 0.15347417 0.99225923]
[0.18233178 0.9401129  0.08688306 0.46821072]
[0.8289892  0.28105226 0.86909151 0.97641657]
[0.84171366 0.4488736  0.37050843 0.48266944]
[0.99754726 0.29104476 0.96903556 0.18828237]
[0. 0. 0. 0.]
[1.         0.         0.         0.51512814]
[0.8636213  0.86375767 0.67488131 0.65987435]
[0.66070137 0.         0.         0.99400298]
[0.         0.39239862 0.6653444  0.53198492]
[0.02023114 0.98343182 0.42723384 0.52627351]
[0.         0.         0.96143969 0.29377208]
[0.48311945 1.         0.         1.        ]
[0.         0.97085173 0.         0.39105143]
[0.82531806 0.219283

In [ ]:
default_model = SVR()
best_model = get_regressor(best_params_HB)

default_model.fit(X_train, y_train)
best_model.fit(X_train, y_train)

default_score = default_model.score(X_test, y_test)
best_score_HB = best_model.score(X_test, y_test)

print('Default model accuracy:', default_score)
print('Best model accuracy:', best_score_HB)

[0.91247415 0.99596633 0.95913823 0.94927731]
Default model accuracy: 0.6652442203874216
Best model accuracy: 0.8588862043868903


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# GreyWolf

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVR

from niapy.problems import Problem
from niapy.task import OptimizationType, Task
from niapy.algorithms.basic import GreyWolfOptimizer


def get_hyperparameters_GWO(x):
    """Get hyperparameters for solution `x`."""
    print(x)
    C=int(5 + x[0] * 10)
    degree=int(5 + x[1] * 10)
    kernels=('poly', 'rbf', 'sigmoid')
    kernel=kernels[int(x[2] * 2)]
    gammas=('scale', 'auto')
    gamma=gammas[int(x[3])]

    params = {
        'C': C,
        'degree': degree,
        'kernel' : kernel,
        'gamma': gamma,
    }

    return params


def get_regressor_GWO(x):
    """Get classifier from solution `x`."""
    params = get_hyperparameters_GWO(x)
    return SVR(**params)

In [ ]:
class SVRHyperparameterOptimization_GWO(Problem):
    def __init__(self, X_train, y_train):
        super().__init__(dimension=4, lower=0, upper=1)
        self.X_train = X_train
        self.y_train = y_train

    def _evaluate(self, x):
        model = get_regressor_GWO(x)
        scores = cross_val_score(model, self.X_train, self.y_train, cv=2, n_jobs=-1)
        return scores.mean()

In [ ]:
problem = SVRHyperparameterOptimization_GWO(X_train, y_train)

# We will be running maximization for 200 iters on `problem`
task = Task(problem, max_iters=200, optimization_type=OptimizationType.MAXIMIZATION)

algorithm = GreyWolfOptimizer(population_size=12, seed=1234)
best_params_GWO, best_accuracy = algorithm.run(task)

print('Best parameters:', get_hyperparameters_GWO(best_params_GWO))

[0.97669977 0.38019574 0.92324623 0.26169242]
[0.31909706 0.11809123 0.24176629 0.31853393]
[0.96407925 0.2636498  0.44100612 0.60987081]
[0.8636213  0.86375767 0.67488131 0.65987435]
[0.7357577  0.22275366 0.17206618 0.87041497]
[0.06013866 0.68368891 0.67123802 0.61101798]
[0.06013731 0.97776927 0.43895163 0.53259502]
[0.00313229 0.25126711 0.85849044 0.42529835]
[0.73581899 0.92204322 0.15347417 0.99225923]
[0.18233178 0.9401129  0.08688306 0.46821072]
[0.8289892  0.28105226 0.86909151 0.97641657]
[0.84171366 0.4488736  0.37050843 0.48266944]
[1.         0.34833099 0.16047346 0.1951096 ]
[1.         0.3849185  0.25199257 1.        ]
[1.         0.09427285 0.30694703 0.99366985]
[0.55099491 0.28795695 0.43938297 0.53038155]
[1.         0.22094345 0.34935178 0.60437614]
[1.         0.18653617 0.36344098 1.        ]
[0.68988784 0.89618788 0.35983827 0.57052387]
[1.         0.38018017 0.         0.79612724]
[0.54554656 0.         0.42645303 0.98659547]
[1.         0.         0.35785742 

In [ ]:
default_model_GWO = SVR()
best_model_GWO = get_regressor_GWO(best_params_GWO)

default_model_GWO.fit(X_train, y_train)
best_model_GWO.fit(X_train, y_train)

default_score = default_model_GWO.score(X_test, y_test)
best_score_GWO = best_model_GWO.score(X_test, y_test)

print('Default model accuracy:', default_score)
print('Best model accuracy:', best_score_GWO)

[0.96701075 0.08764168 1.         0.54306033]
Default model accuracy: 0.6652442203874216
Best model accuracy: 0.8588862043868903


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# PSO

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVR

from niapy.problems import Problem
from niapy.task import OptimizationType, Task
from niapy.algorithms.basic import ParticleSwarmOptimization


def get_hyperparameters_PSO(x):
    """Get hyperparameters for solution `x`."""
    print(x)
    C=int(5 + x[0] * 10)
    degree=int(5 + x[1] * 10)
    kernels=('poly', 'rbf', 'sigmoid')
    kernel=kernels[int(x[2] * 2)]
    gammas=('scale', 'auto')
    gamma=gammas[int(x[3])]

    params = {
        'C': C,
        'degree': degree,
        'kernel' : kernel,
        'gamma': gamma,
    }

    return params


def get_regressor_PSO(x):
    """Get classifier from solution `x`."""
    params = get_hyperparameters_PSO(x)
    return SVR(**params)

In [ ]:
class SVRHyperparameterOptimization_PSO(Problem):
    def __init__(self, X_train, y_train):
        super().__init__(dimension=4, lower=0, upper=1)
        self.X_train = X_train
        self.y_train = y_train

    def _evaluate(self, x):
        model = get_regressor_PSO(x)
        scores = cross_val_score(model, self.X_train, self.y_train, cv=2, n_jobs=-1)
        return scores.mean()

In [ ]:
problem = SVRHyperparameterOptimization_PSO(X_train, y_train)

# We will be running maximization for 200 iters on `problem`
task = Task(problem, max_iters=200, optimization_type=OptimizationType.MAXIMIZATION)

algorithm = ParticleSwarmOptimization(population_size=50, seed=1234)
best_params_PSO, best_accuracy = algorithm.run(task)

print('Best parameters:', get_hyperparameters_PSO(best_params_PSO))

Streaming output truncated to the last 5000 lines.
[0.9486603  0.24744731 0.99605076 1.        ]
[0.94502667 0.93889467 0.33476352 1.        ]
[0.98952408 0.07244821 0.76311969 1.        ]
[0.97990519 0.07827414 0.25130835 1.        ]
[0.98853073 0.         0.37180742 1.        ]
[1.         0.17944373 1.         1.        ]
[0.91408843 0.03853706 0.71189814 1.        ]
[1.         0.09725434 0.64688378 1.        ]
[0.98415862 0.         1.         1.        ]
[0.99694526 0.23834914 0.08027037 1.        ]
[0.98755326 0.         1.         1.        ]
[0.98386135 0.94100532 1.         1.        ]
[0.96466596 0.42784942 0.83897103 1.        ]
[1.        0.2541158 1.        1.       ]
[0.97255075 0.         0.93712273 1.        ]
[0.99559963 0.95410953 1.         1.        ]
[0.98543938 0.05890524 0.95570493 1.        ]
[0.98529833 0.         0.35620036 1.        ]
[0.99271693 0.15958181 0.86565297 1.        ]
[0.99922906 0.08872124 0.33849449 1.        ]
[0.93769764 0.         0.18208643

In [ ]:
default_model_PSO = SVR()
best_model_PSO = get_regressor_PSO(best_params_PSO)

default_model_PSO.fit(X_train, y_train)
best_model_PSO.fit(X_train, y_train)

default_score = default_model_PSO.score(X_test, y_test)
best_score_PSO = best_model_PSO.score(X_test, y_test)

print('Default model accuracy:', default_score)
print('Best model accuracy:', best_score_PSO)

[0.98139944 0.11331412 0.58633719 1.        ]
Default model accuracy: 0.6652442203874216
Best model accuracy: 0.8588862043868903


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


acc 12 pop = 0.8588862043868903
acc

# Result

In [ ]:
print('Best parameters:', get_hyperparameters(best_params_HB))
print('Best parameters:', get_hyperparameters_GWO(best_params_GWO))
print('Best parameters:', get_hyperparameters_PSO(best_params_PSO))

[0.91247415 0.99596633 0.95913823 0.94927731]
Best parameters: {'C': 14, 'degree': 14, 'kernel': 'rbf', 'gamma': 'auto'}
[0.96701075 0.08764168 1.         0.54306033]
Best parameters: {'C': 14, 'degree': 5, 'kernel': 'rbf', 'gamma': 'auto'}
[0.96971124 1.         1.         0.54939289]
Best parameters: {'C': 14, 'degree': 15, 'kernel': 'rbf', 'gamma': 'auto'}


In [ ]:
print(best_score_HB)
print(best_score_GWO)
print(best_score_PSO)

0.8588862043868903
0.8588862043868903
0.8588862043868903
